In [1]:
import os
import cv2
import random
import shutil
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [22]:
train_dir = 'images/train'
testing_dir = 'images/testing'

In [23]:
def data_generators(TRAINING_DIR,TESTING_DIR):

  train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   width_shift_range = 0.3,
                                   height_shift_range = 0.3,
                                   shear_range = 0.3,
                                   zoom_range = 0.3,
                                   horizontal_flip = True,
                                   vertical_flip = True)

  train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size = 64,
                                                    class_mode = 'categorical', 
                                                    target_size = (150, 150))

  testing_datagen = ImageDataGenerator( rescale = 1.0/255.)
    
  testing_generator = testing_datagen.flow_from_directory(TESTING_DIR,
                                                    batch_size = 64,
                                                    class_mode = 'categorical', 
                                                    target_size = (150,150))

  return train_generator, testing_generator

In [24]:
train_generator,testing_generator = data_generators(train_dir, testing_dir)

Found 440 images belonging to 5 classes.
Found 185 images belonging to 5 classes.


In [25]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D

In [26]:
base_model=MobileNet(weights='imagenet',include_top=False,input_shape = (150,150, 3))
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x)
x=Dense(512,activation='relu')(x)
x=Dense(216,activation='relu')(x)
preds=Dense(5,activation='softmax')(x)

In [27]:
len(base_model.layers)

86

In [28]:
for layer in base_model.layers[:50]:
    layer.trainable = False
for layer in base_model.layers[50:86]:
    layer.trainable = True

In [29]:
model_mobilenet=Model(inputs=base_model.input,outputs=preds)

In [35]:
SGD = keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=False)


In [36]:
from tensorflow.keras.optimizers import SGD as LegacySGD

SGD = LegacySGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=False)

ValueError: decay is deprecated in the new Keras optimizer, please check the docstring for valid arguments, or use the legacy optimizer, e.g., tf.keras.optimizers.legacy.SGD.

In [37]:
model_mobilenet.compile(optimizer=SGD,loss='categorical_crossentropy',metrics=['accuracy'])

In [38]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('val_accuracy') > 0.85:
            print("\nReached 85% accuracy so cancelling training!")
            self.model.stop_training = True

In [39]:
callbacks = myCallback()

In [41]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('val_acc') > 0.85:
            print("\nReached 85% validation accuracy, so cancelling training!")
            self.model.stop_training = True


In [43]:
history = model_mobilenet.fit(train_generator,
                    epochs=50,
                          verbose=1,callbacks=[callbacks])

Epoch 1/50
7/7 [==============================] - ETA: 0s - loss: 1.3033 - accuracy: 0.4886

TypeError: '>' not supported between instances of 'NoneType' and 'float'

In [ ]:
results = model_mobilenet.evaluate(testing_generator, batch_size=128)